# Walkthrough on Hyperparameter Optimization using Weights and Biases

## Clone and Install Anomalib

In [1]:
!git clone https://github.com/openvinotoolkit/anomalib.git

Cloning into 'anomalib'...
remote: Enumerating objects: 21618, done.
remote: Counting objects: 100% (3157/3157), done.
remote: Compressing objects: 100% (1047/1047), done.
remote: Total 21618 (delta 1785), reused 2905 (delta 1622), pack-reused 18461
Receiving objects: 100% (21618/21618), 49.64 MiB | 35.03 MiB/s, done.
Resolving deltas: 100% (12658/12658), done.


In [3]:
%cd anomalib

/content/anomalib


In [ ]:
%pip install .

> Note: Restart Runtime if promted by clicking the button at the end of the install logs

## Download and Setup Dataset

In [2]:
%cd /content/anomalib/

/content/anomalib


In [ ]:
!wget https://openvinotoolkit.github.io/anomalib/_downloads/3f2af1d7748194b18c2177a34c03a2c4/hazelnut_toy.zip

In [7]:
!mkdir datasets && unzip hazelnut_toy.zip -d datasets/ > /dev/null

## Creating training configuration

Since we are using our [custom dataset](https://openvinotoolkit.github.io/anomalib/how_to_guides/train_custom_data.html) we need to modify the default configuration file. The following configuration is based on the one available here `anomalib/anomalib/models/stfpm/config.yaml`

In [13]:
folder_stfpm = """dataset:
  name: hazelnut
  format: folder
  path: /content/anomalib/datasets/hazelnut_toy
  normal_dir: good # name of the folder containing normal images.
  abnormal_dir: colour # name of the folder containing abnormal images.
  normal_test_dir: null # name of the folder containing normal test images.
  task: segmentation # classification or segmentation
  mask: /content/anomalib/datasets/hazelnut_toy/mask/colour # optional
  extensions: null
  split_ratio: 0.2  # ratio of the normal images that will be used to create a test split
  image_size: 256
  train_batch_size: 32
  test_batch_size: 32
  num_workers: 8
  transform_config:
    train: null
    val: null
  create_validation_set: false
  tiling:
    apply: false
    tile_size: null
    stride: null
    remove_border_count: 0
    use_random_tiling: False
    random_tile_count: 16

model:
  name: stfpm
  backbone: resnet18
  layers:
    - layer1
    - layer2
    - layer3
  lr: 0.4
  momentum: 0.9
  weight_decay: 0.0001
  early_stopping:
    patience: 3
    metric: pixel_AUROC
    mode: max
  normalization_method: min_max # options: [null, min_max, cdf]

metrics:
  image:
    - F1Score
    - AUROC
  pixel:
    - F1Score
    - AUROC
  threshold:
    image_default: 0
    pixel_default: 0
    adaptive: true

visualization:
  show_images: False # show images on the screen
  save_images: False # save images to the file system
  log_images: False # log images to the available loggers (if any)
  image_save_path: null # path to which images will be saved
  mode: full # options: ["full", "simple"]

project:
  seed: 0
  path: ./results

logging:
  logger: [] # options: [comet, tensorboard, wandb, csv] or combinations.
  log_graph: false # Logs the model graph to respective logger.

optimization:
  export_mode: null #options: onnx, openvino
# PL Trainer Args. Don't add extra parameter here.
trainer:
  accelerator: auto # <"cpu", "gpu", "tpu", "ipu", "hpu", "auto">
  accumulate_grad_batches: 1
  amp_backend: native
  auto_lr_find: false
  auto_scale_batch_size: false
  auto_select_gpus: false
  benchmark: false
  check_val_every_n_epoch: 1
  default_root_dir: null
  detect_anomaly: false
  deterministic: false
  devices: 1
  enable_checkpointing: true
  enable_model_summary: true
  enable_progress_bar: true
  fast_dev_run: false
  gpus: null # Set automatically
  gradient_clip_val: 0
  ipus: null
  limit_predict_batches: 1.0
  limit_test_batches: 1.0
  limit_train_batches: 1.0
  limit_val_batches: 1.0
  log_every_n_steps: 50
  log_gpu_memory: null
  max_epochs: 100
  max_steps: -1
  max_time: null
  min_epochs: null
  min_steps: null
  move_metrics_to_cpu: false
  multiple_trainloader_mode: max_size_cycle
  num_nodes: 1
  num_processes: null
  num_sanity_val_steps: 0
  overfit_batches: 0.0
  plugins: null
  precision: 32
  profiler: null
  reload_dataloaders_every_n_epochs: 0
  replace_sampler_ddp: true
  strategy: null
  sync_batchnorm: false
  tpu_cores: null
  track_grad_norm: -1
  val_check_interval: 1.0
"""

with open("model_config.yaml", "w", encoding="utf8") as f:
    f.writelines(folder_stfpm)

## Create sweep configuration

The following configuration file is based on the one available at `anomalib/tools/hpo/configs/stfpm.yaml`. For this example we will use the STFPM model.

In [32]:
sweep_config = """observation_budget: 10
method: bayes
metric:
  name: pixel_AUROC
  goal: maximize
parameters:
  dataset:
    category: capsule
    image_size:
      values: [128, 256]
  model:
    backbone:
      values: [resnet18, wide_resnet50_2]
    lr:
      min: 0.1
      max: 0.9
    momentum:
      min: 0.1
      max: 0.9
"""

with open("sweep_config.yaml", "w", encoding="utf8") as f:
    f.writelines(sweep_config)

In [ ]:
!python ./tools/hpo/sweep.py --model stfpm --model_config model_config.yaml --sweep_config sweep_config.yaml

While only a few parameters were shown in this example, you can call HPO on any of the parameters defined in the `model` and `dataset` section of the model configuration file.

Congratulations 🎉 You have now successfully optimized a model on your dataset.

To go into more detail, refer our documentation on [hyperparameter optimization](https://openvinotoolkit.github.io/anomalib/tutorials/hyperparameter_optimization.html).